In [1]:
import numpy as np
import pandas as pd

In [2]:
excel_name = '成本核算数据源配置表.xlsx'
sheet_name = {
    'source': {
        'revenue': '收入',
        'spend':'投放费',
        'timesheet':'人员项目工时',
        'employee':'人员工资奖金六险一金',
        'cloud':'云服务费',
        'technical':'技术服务费',
        'operation':'运营费用',
        'bjdepartment':'北京天天部门费用',
        'cddepartment':'成都天天部门费用',
    },
    'config': {
        'employee':'人员费用配置表',
        'project':'项目费用配置表'
    }
}

In [3]:
timesheet = pd.read_excel(excel_name, sheet_name=sheet_name['source']['timesheet'])
employee = pd.read_excel(excel_name, sheet_name=sheet_name['source']['employee'])
bjdepartment = pd.read_excel(excel_name, sheet_name=sheet_name['source']['bjdepartment'])
cddepartment = pd.read_excel(excel_name, sheet_name=sheet_name['source']['cddepartment'])
employee_config = pd.read_excel(excel_name, sheet_name=sheet_name['config']['employee'])

In [4]:
timesheet['项目组'] = timesheet['项目组'].fillna('')
timesheet.fillna(0, inplace=True)
timesheet.set_index('人员', inplace = True)
# timesheet

In [5]:
employee['项目组'] = employee['项目组'].fillna('')
employee.fillna(0, inplace=True)
employee.set_index('人员', inplace=True)
# employee

In [6]:
bjdepartment.fillna(0, inplace=True)
bjdepartment.set_index('管理费用', inplace=True)
cddepartment.fillna(0, inplace=True)
cddepartment.set_index('管理费用', inplace=True)
# bjdepartment

In [7]:
employee_config.columns = list(employee_config.columns[0:6]) + list(employee_config.loc[0])[6:]
employee_config.drop([0,1], inplace=True)
# employee_config.reset_index(inplace=True)
employee_config.set_index('人员', inplace=True)
employee_config[['项目组', '参与了哪些直接项目组项目']] = employee_config[['项目组', '参与了哪些直接项目组项目']].fillna('')
employee_config.fillna(0, inplace=True)
# employee_config

In [8]:
# 连接表数据，合并基础数据和配置
employee_fee = employee_config.copy()
employee['六险一金'] = employee['五险一金'] + employee['补充医疗']
employee_fee_merge= pd.merge(employee_config, employee[['工资', '六险一金']], left_index=True, right_index=True, how='left')
# employee_fee_merge

In [9]:
# 项目列表 
# 公摊项目
项目列表 = timesheet.columns.values[3:]
公摊项目 = '公摊项目'

In [10]:
# 研发部门费用计算

# 中间变量
工资 = timesheet[项目列表].copy()
六险一金 = timesheet[项目列表].copy()
工资社保 = timesheet[项目列表].copy()
A1部门分摊费用 = timesheet[项目列表].copy()
A2部门分摊费用 = timesheet[项目列表].copy()
A3部门分摊费用 = timesheet[项目列表].copy()
A项目美术部门分摊费用 = timesheet[项目列表].copy()
A项目程序部门分摊费用 = timesheet[项目列表].copy()
A项目公摊分摊费用 = timesheet[项目列表].copy()
H1部门分摊费用 = timesheet[项目列表].copy()
H2部门分摊费用 = timesheet[项目列表].copy()
H项目公摊分摊费用 = timesheet[项目列表].copy()
T1部门分摊费用 = timesheet[项目列表].copy()
T2部门分摊费用 = timesheet[项目列表].copy()
T项目美术部门分摊费用 = timesheet[项目列表].copy()
T项目公摊分摊费用 = timesheet[项目列表].copy()
部门其他费用 = timesheet[项目列表].copy()

# 数据源获取
数据源_研发人员工资费用 = employee_fee_merge['研发人员工资'] * employee_fee_merge['工资']
数据源_研发人员六险一金费用 = employee_fee_merge['研发人员六险一金'] * employee_fee_merge['六险一金']
数据源_A1部门费用 = (employee_fee_merge['A1部门费用'] / employee_fee_merge['A1部门费用'].sum())  * bjdepartment['A1项目']['   合   计']
数据源_A2部门费用 = (employee_fee_merge['A2部门费用'] / employee_fee_merge['A2部门费用'].sum())  * bjdepartment['A2项目']['   合   计']
数据源_A3部门费用 = (employee_fee_merge['A3部门费用'] / employee_fee_merge['A3部门费用'].sum())  * bjdepartment['A3项目']['   合   计']
数据源_A项目美术部门费用= (employee_fee_merge['A项目美术部门费用'] / employee_fee_merge['A项目美术部门费用'].sum())  * bjdepartment['A项目美术']['   合   计']
数据源_A项目程序部门费用 = (employee_fee_merge['A项目程序部门费用'] / employee_fee_merge['A项目程序部门费用'].sum())  * bjdepartment['A项目程序']['   合   计']
数据源_H1部门费用= (employee_fee_merge['H1部门费用'] / employee_fee_merge['H1部门费用'].sum())  * bjdepartment['H1项目']['   合   计']
数据源_H2部门费用= (employee_fee_merge['H2部门费用'] / employee_fee_merge['H2部门费用'].sum())  * bjdepartment['H2项目']['   合   计']
数据源_T1部门费用= (employee_fee_merge['T1部门费用'] / employee_fee_merge['T1部门费用'].sum())  * cddepartment['T1项目']['   合   计']
数据源_T2部门费用= (employee_fee_merge['T2部门费用'] / employee_fee_merge['T2部门费用'].sum())  * cddepartment['T2项目']['   合   计']
数据源_T项目美术部门费用= (employee_fee_merge['T项目美术部门费用'] / employee_fee_merge['T项目美术部门费用'].sum())  * cddepartment['T项目美术']['   合   计']
for 项目名 in 项目列表:
    工资[项目名] = 工资[项目名] * 数据源_研发人员工资费用 / 100
    六险一金[项目名] = 六险一金[项目名] * 数据源_研发人员六险一金费用 / 100
    工资社保[项目名] = 六险一金[项目名] + 工资[项目名]
    A1部门分摊费用[项目名] = A1部门分摊费用[项目名] * 数据源_A1部门费用 / 100
    A2部门分摊费用[项目名] = A2部门分摊费用[项目名] * 数据源_A2部门费用 / 100
    A3部门分摊费用[项目名] = A3部门分摊费用[项目名] * 数据源_A3部门费用 / 100
    A项目美术部门分摊费用[项目名] = A项目美术部门分摊费用[项目名] * 数据源_A项目美术部门费用 / 100
    A项目程序部门分摊费用[项目名] = A项目程序部门分摊费用[项目名] * 数据源_A项目程序部门费用 / 100
    H1部门分摊费用[项目名] = H1部门分摊费用[项目名] * 数据源_H1部门费用 / 100
    H2部门分摊费用[项目名] = H2部门分摊费用[项目名] * 数据源_H2部门费用 / 100
    T1部门分摊费用[项目名] = T1部门分摊费用[项目名] * 数据源_T1部门费用 / 100
    T2部门分摊费用[项目名] = T2部门分摊费用[项目名] * 数据源_T2部门费用 / 100
    T项目美术部门分摊费用[项目名] = T项目美术部门分摊费用[项目名] * 数据源_T项目美术部门费用 / 100

# 公摊费用各个支费用获取
# 公摊费用二次分摊
数据源_A项目公摊费用汇总 = (A1部门分摊费用[公摊项目] + A2部门分摊费用[公摊项目] + A3部门分摊费用[公摊项目] + A项目美术部门分摊费用[公摊项目] + A项目程序部门分摊费用[公摊项目]).sum()
数据源_A项目公摊费用 = employee_fee_merge['A项目公摊费用'] / employee_fee_merge['A项目公摊费用'].sum() * 数据源_A项目公摊费用汇总
数据源_H项目公摊费用汇总 = (H1部门分摊费用[公摊项目] + H1部门分摊费用[公摊项目]).sum()
数据源_H项目公摊费用 = employee_fee_merge['H项目公摊费用'] / employee_fee_merge['H项目公摊费用'].sum() * 数据源_H项目公摊费用汇总
数据源_T项目公摊费用汇总 = (T1部门分摊费用[公摊项目] + T2部门分摊费用[公摊项目] + T项目美术部门分摊费用[公摊项目]).sum()
数据源_T项目公摊费用 = employee_fee_merge['T项目公摊费用'] / employee_fee_merge['T项目公摊费用'].sum() * 数据源_T项目公摊费用汇总
for 项目名 in 项目列表:
    A项目公摊分摊费用[项目名] = A项目公摊分摊费用[项目名] * 数据源_A项目公摊费用 / 100
    H项目公摊分摊费用[项目名] = H项目公摊分摊费用[项目名] * 数据源_H项目公摊费用 / 100
    T项目公摊分摊费用[项目名] = T项目公摊分摊费用[项目名] * 数据源_T项目公摊费用 / 100
    部门其他费用[项目名] = A项目公摊分摊费用[项目名] + H项目公摊分摊费用[项目名] + T项目公摊分摊费用[项目名]

部门其他费用

,纸牌-Collection Fish,纸牌-Solitaire 3D Fish,纸牌-Solitaire Dragons,纸牌-Solitaire Fish,纸牌-Spider Fish,纸牌-Tripeaks Fish,纸牌-Solitaire Farm Friends,纸牌-Solitaire Garden,纸牌-Solitaire Resort,纸牌-Solitaire Club,...,智力-Breaker Fun 2,智力-Tile Farm,智力-Tile Match,智力-Sort Fun,智力-Tile Travel 3D,智力-Merge Cooking,纸牌-Tripeaks Lucky,智力-Dice Merge,智力-Tile Artist,公摊项目
人员,,,,,,,,,,,,,,,,,,,,,
员工a1,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00000
员工a2,0.0,52.77415,0.0,47.97650,0.0,43.17885,0.0,38.38120,0.0,43.17885,...,0.0,43.17885,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,62.36945
员工a3,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00000
员工a4,0.0,0.00000,0.0,52.77415,0.0,47.97650,0.0,43.17885,0.0,38.38120,...,0.0,47.97650,0.0,43.17885,0.0,0.00000,0.0,0.0,0.0,62.36945
员工a5,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00000
员工a6,0.0,0.00000,0.0,0.00000,0.0,52.77415,0.0,47.97650,0.0,43.17885,...,0.0,52.77415,0.0,47.97650,0.0,43.17885,0.0,0.0,0.0,62.36945
员工a7,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00000
员工a8,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00000
员工a9,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.00000,...,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00000


In [11]:
# 市场部门费用计算

# 中间变量
北京天天市场投放部工资六险一金分摊费用 = timesheet[项目列表].copy()
北京天天市场投放部部门其他费用分摊费用 = timesheet[项目列表].copy()
北京天天市场素材部工资六险一金分摊费用 = timesheet[项目列表].copy()
北京天天市场素材部部门其他费用分摊费用 = timesheet[项目列表].copy()
北京天天市场部分摊公摊费用 = timesheet[项目列表].copy()
成都天天市场部工资六险一金分摊费用 = timesheet[项目列表].copy()
成都天天市场部部门其他费用分摊费用 =  timesheet[项目列表].copy()
成都天天市场部分摊公摊费用 = timesheet[项目列表].copy()
分摊市场人员费用 = timesheet[项目列表].copy()

# 数据源获取
数据源_北京天天市场投放部工资六险一金人员费用 = employee_fee_merge['北京天天市场投放部工资六险一金'] * (employee_fee_merge['工资'] + employee_fee_merge['六险一金']) 
数据源_北京天天市场投放部部门其他费用 = (employee_fee_merge['北京天天市场投放部部门其他费用'] / employee_fee_merge['北京天天市场投放部部门其他费用'].sum())  * bjdepartment['市场投放部']['   合   计']
数据源_北京天天市场素材部部门其他费用 = (employee_fee_merge['北京天天市场素材部部门其他费用'] / employee_fee_merge['北京天天市场素材部部门其他费用'].sum())  * bjdepartment['市场素材部']['   合   计']
数据源_北京天天市场素材部工资六险一金人员费用 = employee_fee_merge['北京天天市场素材部工资六险一金'] * (employee_fee_merge['工资'] + employee_fee_merge['六险一金']) 
数据源_成都天天市场部工资六险一金人员费用 = employee_fee_merge['成都天天市场部工资六险一金'] * (employee_fee_merge['工资'] + employee_fee_merge['六险一金']) 
数据源_成都天天市场部部门其他费用 = (employee_fee_merge['成都天天市场部部门其他费用'] / employee_fee_merge['成都天天市场部部门其他费用'].sum())  * cddepartment['市场部']['   合   计']

for 项目名 in 项目列表:
    北京天天市场投放部工资六险一金分摊费用[项目名] = 北京天天市场投放部工资六险一金分摊费用[项目名] * 数据源_北京天天市场投放部工资六险一金人员费用 / 100
    北京天天市场投放部部门其他费用分摊费用[项目名] = 北京天天市场投放部部门其他费用分摊费用[项目名] * 数据源_北京天天市场投放部部门其他费用 / 100
    北京天天市场素材部工资六险一金分摊费用[项目名] = 北京天天市场素材部工资六险一金分摊费用[项目名] * 数据源_北京天天市场素材部部门其他费用 / 100
    北京天天市场素材部部门其他费用分摊费用[项目名] = 北京天天市场素材部部门其他费用分摊费用[项目名] * 数据源_北京天天市场素材部工资六险一金人员费用 / 100
    成都天天市场部工资六险一金分摊费用[项目名] = 成都天天市场部工资六险一金分摊费用[项目名] * 数据源_北京天天市场素材部工资六险一金人员费用 / 100
    成都天天市场部部门其他费用分摊费用[项目名] = 成都天天市场部部门其他费用分摊费用[项目名] * 数据源_成都天天市场部部门其他费用 / 100
    
# 公摊费用各个支费用获取
# 公摊费用二次分摊
数据源_北京天天市场部公摊费用汇总 = (北京天天市场投放部工资六险一金分摊费用[公摊项目] + 北京天天市场投放部部门其他费用分摊费用[公摊项目] + 北京天天市场素材部工资六险一金分摊费用[公摊项目] + 北京天天市场素材部部门其他费用分摊费用[公摊项目]).sum()
数据源_北京天天市场部公摊费用 = employee_fee_merge['北京天天分摊市场部门费用'] / employee_fee_merge['北京天天分摊市场部门费用'].sum() * 数据源_北京天天市场部公摊费用汇总
数据源_成都天天市场部公摊费用汇总 = (成都天天市场部工资六险一金分摊费用[公摊项目] + 成都天天市场部部门其他费用分摊费用[公摊项目]).sum()
数据源_成都天天市场部公摊费用 = employee_fee_merge['成都天天分摊市场部门费用'] / employee_fee_merge['成都天天分摊市场部门费用'].sum() * 数据源_成都天天市场部公摊费用汇总
for 项目名 in 项目列表:
    北京天天市场部分摊公摊费用[项目名] = 北京天天市场部分摊公摊费用[项目名] * 数据源_北京天天市场部公摊费用 / 100
    成都天天市场部分摊公摊费用[项目名] = 成都天天市场部分摊公摊费用[项目名] * 数据源_成都天天市场部公摊费用 / 100
    分摊市场人员费用[项目名] = 北京天天市场部分摊公摊费用[项目名] + 成都天天市场部分摊公摊费用[项目名]
分摊市场人员费用

,纸牌-Collection Fish,纸牌-Solitaire 3D Fish,纸牌-Solitaire Dragons,纸牌-Solitaire Fish,纸牌-Spider Fish,纸牌-Tripeaks Fish,纸牌-Solitaire Farm Friends,纸牌-Solitaire Garden,纸牌-Solitaire Resort,纸牌-Solitaire Club,...,智力-Breaker Fun 2,智力-Tile Farm,智力-Tile Match,智力-Sort Fun,智力-Tile Travel 3D,智力-Merge Cooking,纸牌-Tripeaks Lucky,智力-Dice Merge,智力-Tile Artist,公摊项目
人员,,,,,,,,,,,,,,,,,,,,,
员工a1,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a2,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a3,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a4,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a5,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a6,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a7,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a8,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
员工a9,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [12]:
# 职能部门费用计算

# 中间变量
北京人事行政部费用 = timesheet[项目列表].copy()
成都天天人事行政部费用 = timesheet[项目列表].copy()
财务和总裁办费用 = timesheet[项目列表].copy()
分摊职能人员费用 = timesheet[项目列表].copy()

数据源_北京天天人事行政部费用 = (employee_fee_merge['北京天天人事行政部费用'] / employee_fee_merge['北京天天人事行政部费用'].sum()) * bjdepartment['人事行政部']['   合   计']
数据源_成都天天人事行政部费用 = (employee_fee_merge['成都天天人事行政部费用'] / employee_fee_merge['成都天天人事行政部费用'].sum()) * cddepartment['人事行政部']['   合   计']
数据源_财务和总裁办费用 = (employee_fee_merge['财务和总裁办费用'] / employee_fee_merge['财务和总裁办费用'].sum()) * (bjdepartment['财务部']['   合   计'] + bjdepartment['总裁办']['   合   计'])

for 项目名 in 项目列表:
    北京人事行政部费用[项目名] = 北京人事行政部费用[项目名] * 数据源_北京天天人事行政部费用 / 100
    成都天天人事行政部费用[项目名] = 成都天天人事行政部费用[项目名] * 数据源_成都天天人事行政部费用 / 100
    财务和总裁办费用[项目名] = 财务和总裁办费用[项目名] * 数据源_财务和总裁办费用 / 100
    分摊职能人员费用 = 北京人事行政部费用[项目名] + 成都天天人事行政部费用[项目名] + 财务和总裁办费用[项目名]
    
分摊职能人员费用

人员
员工a1         0.000000
员工a2       177.149143
员工a3         0.000000
员工a4       177.149143
员工a5         0.000000
员工a6       177.149143
员工a7         0.000000
员工a8         0.000000
员工a9         0.000000
员工a10        0.000000
员工h1         0.000000
员工h2       177.149143
员工h3         0.000000
员工h4       177.149143
员工t1         0.000000
员工t2       292.082143
员工t3         0.000000
员工t4       292.082143
员工t5         0.000000
员工t6         0.000000
市场投放员工1      0.000000
市场投放员工2      0.000000
市场素材员工1      0.000000
市场素材员工2      0.000000
市场员工1        0.000000
市场员工2        0.000000
市场员工3        0.000000
邱鑫           0.000000
邸飞           0.000000
霍楠           0.000000
孙悦           0.000000
满静           0.000000
许阳清          0.000000
董勃言          0.000000
邱鑫1          0.000000
邸飞1          0.000000
霍楠1          0.000000
许阳清1         0.000000
董勃言1         0.000000
Name: 公摊项目, dtype: float64

In [13]:
# 中台部门费用分摊计算

# 中间变量
中台测试和中台美术人员费用 = timesheet.copy()
中台测试和中台美术公摊费用 = timesheet.copy()
中台研发和研发中台人员费用 = timesheet.copy()
中台研发和研发中台公摊费用 = timesheet.copy()
美术中台人员费用 = timesheet.copy()
美术中台公摊费用 = timesheet.copy()
分摊公摊项目费用 = timesheet.copy()

# 数据源获取
数据源_中台测试和中台美术工资六险一金人员费用 = employee_fee_merge['中台测试和中台美术人员费用'] * (employee_fee_merge['工资'] + employee_fee_merge['六险一金'])
数据源_中台测试和中台美术部门其他费用 = (employee_fee_merge['中台测试和中台美术人员费用'] / employee_fee_merge['中台测试和中台美术人员费用'].sum())  * (bjdepartment['中台测试部']['   合   计'] + bjdepartment['中台美术部']['   合   计'])
数据源_中台研发和研发中台工资六险一金人员费用 = employee_fee_merge['中台研发和研发中台人员费用'] * (employee_fee_merge['工资'] + employee_fee_merge['六险一金'])
数据源_中台研发和研发中台部门其他费用 = (employee_fee_merge['中台研发和研发中台人员费用'] / employee_fee_merge['中台研发和研发中台人员费用'].sum())  * (bjdepartment['中台研发部']['   合   计'] + cddepartment['研发中台部']['   合   计'])
数据源_美术中台工资六险一金人员费用 = employee_fee_merge['美术中台人员费用'] * (employee_fee_merge['工资'] + employee_fee_merge['六险一金'])
数据源_美术中台部门其他费用 =  (employee_fee_merge['美术中台人员费用'] / employee_fee_merge['美术中台人员费用'].sum())  * cddepartment['美术中台部']['   合   计']
for 项目名 in 项目列表:
    中台测试和中台美术人员费用[项目名] = 中台测试和中台美术人员费用[项目名] * (数据源_中台测试和中台美术工资六险一金人员费用 + 数据源_中台测试和中台美术部门其他费用) / 100
    中台研发和研发中台人员费用[项目名] = 中台研发和研发中台人员费用[项目名] * (数据源_中台研发和研发中台工资六险一金人员费用 + 数据源_中台研发和研发中台部门其他费用) / 100
    美术中台人员费用[项目名] = 美术中台人员费用[项目名] * (数据源_美术中台工资六险一金人员费用 + 数据源_美术中台部门其他费用) / 100
    
# 公摊费用各个支费用计算
# 公摊费用二次分摊
数据源_中台测试和中台美术人员费用汇总 = (中台测试和中台美术人员费用[公摊项目]).sum()
数据源_中台测试和中台美术人员费用 = employee_fee_merge['中台测试和中台美术公摊费用'] / employee_fee_merge['中台测试和中台美术公摊费用'].sum() * 数据源_中台测试和中台美术人员费用汇总
数据源_中台研发和研发中台人员费用汇总 = (中台研发和研发中台人员费用[公摊项目]).sum()
数据源_中台研发和研发中台人员费用 = employee_fee_merge['中台研发和研发中台公摊费用'] / employee_fee_merge['中台研发和研发中台公摊费用'].sum() * 数据源_中台研发和研发中台人员费用汇总
数据源_美术中台人员费用汇总 = (美术中台人员费用[公摊项目]).sum()
数据源_美术中台人员费用 = employee_fee_merge['美术中台公摊费用'] / employee_fee_merge['美术中台公摊费用'].sum() * 数据源_美术中台人员费用汇总
for 项目名 in 项目列表:
    中台测试和中台美术公摊费用[项目名] = 中台测试和中台美术公摊费用[项目名] * 数据源_中台测试和中台美术人员费用 / 100
    中台研发和研发中台公摊费用[项目名] = 中台研发和研发中台公摊费用[项目名] * 数据源_中台研发和研发中台人员费用 / 100
    美术中台公摊费用[项目名] = 美术中台公摊费用[项目名] * 数据源_美术中台人员费用 / 100
    分摊公摊项目费用[项目名] = 中台测试和中台美术公摊费用[项目名] + 中台研发和研发中台公摊费用[项目名] + 美术中台公摊费用[项目名]

分摊公摊项目费用


,主体,部门,项目组,纸牌-Collection Fish,纸牌-Solitaire 3D Fish,纸牌-Solitaire Dragons,纸牌-Solitaire Fish,纸牌-Spider Fish,纸牌-Tripeaks Fish,纸牌-Solitaire Farm Friends,...,智力-Breaker Fun 2,智力-Tile Farm,智力-Tile Match,智力-Sort Fun,智力-Tile Travel 3D,智力-Merge Cooking,纸牌-Tripeaks Lucky,智力-Dice Merge,智力-Tile Artist,公摊项目
人员,,,,,,,,,,,,,,,,,,,,,
员工a1,北京天天,A1,A项目组,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
员工a2,北京天天,A1,A项目组,0.0,204.350064,0.0,185.772786,0.0,167.195507,0.0,...,0.0,167.195507,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,241.504621
员工a3,北京天天,A2,A项目组,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
员工a4,北京天天,A2,A项目组,0.0,0.000000,0.0,204.350064,0.0,185.772786,0.0,...,0.0,185.772786,0.0,167.195507,0.0,0.000000,0.0,0.0,0.0,241.504621
员工a5,北京天天,A3,A项目组,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
员工a6,北京天天,A3,A项目组,0.0,0.000000,0.0,0.000000,0.0,204.350064,0.0,...,0.0,204.350064,0.0,185.772786,0.0,167.195507,0.0,0.0,0.0,241.504621
员工a7,北京天天,A项目美术,,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
员工a8,北京天天,A项目美术,,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
员工a9,北京天天,A项目程序,,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
